In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label
from collections import deque

def generate_landscape(grid_size=(50, 50), terrain_types=5):
    """生成一個隨機地形 (2D grid)。"""
    return np.random.randint(0, terrain_types, size=grid_size)

def visualize_landscape(landscape, filename):
    """將地形視覺化並存檔。"""
    plt.figure(figsize=(6, 6))
    plt.imshow(landscape, cmap='terrain', origin='upper')
    plt.colorbar(label="Terrain Type")
    plt.axis('off')
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

def calculate_region_match(landscape, region_coords, target_terrain):
    """
    計算指定區域中，目標地形 (target_terrain) 的覆蓋比例。
    region_coords = (row_start, row_end, col_start, col_end)
    """
    region = landscape[region_coords[0]:region_coords[1], region_coords[2]:region_coords[3]]
    match_score = np.sum(region == target_terrain) / region.size
    return match_score

def calculate_river_connectivity(landscape, river_type=0):
    """
    檢查是否存在一條「從地圖左側一直連通到右側」的河流。
    回傳值介於 [0, 1]，1 代表確實有貫穿地圖的河流，0 代表沒有。
    """
    rows, cols = landscape.shape
    visited = np.zeros_like(landscape, dtype=bool)
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    queue = deque()

    # 先將「左側所有屬於河流」的座標放入 queue
    for r in range(rows):
        if landscape[r, 0] == river_type:
            queue.append((r, 0))
            visited[r, 0] = True

    # BFS (或 DFS) 搜索，看能否到達右側
    found_path = False
    while queue:
        r, c = queue.popleft()
        # 若抵達最右側，就算成功貫穿
        if c == cols - 1:
            found_path = True
            break
        
        for dr, dc in directions:
            rr, cc = r + dr, c + dc
            if 0 <= rr < rows and 0 <= cc < cols:
                if not visited[rr, cc] and landscape[rr, cc] == river_type:
                    visited[rr, cc] = True
                    queue.append((rr, cc))

    return 1.0 if found_path else 0.0

def fitness_function(landscape, terrain_types=5):
    """
    綜合適應度函數：
    1) 北部是否為雪山 (type=4)
    2) 南部是否為草原 (type=1)
    3) 是否有河流 (type=0) 橫越地圖
    """
    # 1) 北部雪山評分
    north_region_coords = (0, landscape.shape[0] // 3, 0, landscape.shape[1])
    north_match_score = calculate_region_match(landscape, north_region_coords, target_terrain=4)

    # 2) 南部草原評分
    south_region_coords = (2 * landscape.shape[0] // 3, landscape.shape[0], 0, landscape.shape[1])
    south_match_score = calculate_region_match(landscape, south_region_coords, target_terrain=1)

    # 3) 河流貫穿性評分
    river_connectivity_score = calculate_river_connectivity(landscape, river_type=0)

    # 調整權重，若很想要河流貫穿，可以提高其權重
    total_fitness = (0.4 * north_match_score +
                     0.4 * south_match_score +
                     0.8* river_connectivity_score)

    return total_fitness

def crossover(parent1, parent2):
    """簡單的單點遮罩交配：隨機將 parent1 或 parent2 的像素拿來生成 child。"""
    mask = np.random.randint(0, 2, size=parent1.shape).astype(bool)
    child = np.where(mask, parent1, parent2)
    return child

def mutate(landscape, mutation_rate=0.01, terrain_types=5):
    """變異操作：以 mutation_rate 的機率，將像素替換為隨機地形。"""
    mutation_mask = np.random.rand(*landscape.shape) < mutation_rate
    new_values = np.random.randint(0, terrain_types, size=landscape.shape)
    landscape[mutation_mask] = new_values[mutation_mask]
    return landscape

def initialize_population(pop_size, grid_size, terrain_types):
    """初始化種群：生成 pop_size 張隨機地形圖。"""
    return [generate_landscape(grid_size, terrain_types) for _ in range(pop_size)]

def evolve_population(population, fitnesses, terrain_types, mutation_rate):
    """進化過程：選擇、交配 (crossover)、變異 (mutation)。"""
    # 1) 根據適應度，挑選前 50% 個體 (簡單淘汰式選擇)
    sorted_indices = np.argsort(fitnesses)[::-1]
    top_half = [population[i] for i in sorted_indices[:len(population) // 2]]

    # 2) 進行交配 + 變異，補足新一代人口
    new_population = []
    while len(new_population) < len(population):
        parents = np.random.choice(len(top_half), size=2, replace=False)
        parent1, parent2 = top_half[parents[0]], top_half[parents[1]]
        child = crossover(parent1, parent2)
        child = mutate(child, mutation_rate, terrain_types)
        new_population.append(child)

    return new_population

def main_evolution(
    output_folder='output',
    num_generations=300,      # 減少迭代次數做測試
    pop_size=1000,             # 增加族群大小以提高多樣性
    grid_size=(50, 50),
    terrain_types=5,
    mutation_rate=0.02        # 稍微提高突變率，看是否能加速跳出解
):
    """透過簡單演化算法，不斷優化地形。"""
    os.makedirs(output_folder, exist_ok=True)

    # 初始化種群
    population = initialize_population(pop_size, grid_size, terrain_types)

    for generation in range(num_generations):
        # 計算適應度
        fitnesses = [fitness_function(ind, terrain_types) for ind in population]
        best_fitness = max(fitnesses)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.3f}")

        # 若進到最後幾代，存下最優地形看看成果
        if generation >= num_generations - 10:
            best_index = np.argmax(fitnesses)
            best_landscape = population[best_index]
            filename = os.path.join(
                output_folder,
                f'best_landscape_gen_{generation+1}_fitness_{fitnesses[best_index]:.3f}.png'
            )
            visualize_landscape(best_landscape, filename)

        # 演化到下一代
        population = evolve_population(population, fitnesses, terrain_types, mutation_rate)

    print("Evolution Complete!")

if __name__ == '__main__':
    main_evolution()


Generation 1: Best Fitness = 0.195
Generation 2: Best Fitness = 0.193
Generation 3: Best Fitness = 0.192
Generation 4: Best Fitness = 0.199
Generation 5: Best Fitness = 0.200
Generation 6: Best Fitness = 0.208
Generation 7: Best Fitness = 0.213
Generation 8: Best Fitness = 0.217
Generation 9: Best Fitness = 0.223
Generation 10: Best Fitness = 0.224
Generation 11: Best Fitness = 0.227
Generation 12: Best Fitness = 0.234
Generation 13: Best Fitness = 0.237
Generation 14: Best Fitness = 0.242
Generation 15: Best Fitness = 0.250
Generation 16: Best Fitness = 0.256
Generation 17: Best Fitness = 0.261
Generation 18: Best Fitness = 0.256
Generation 19: Best Fitness = 0.261
Generation 20: Best Fitness = 0.264
Generation 21: Best Fitness = 0.270
Generation 22: Best Fitness = 0.275
Generation 23: Best Fitness = 0.279
Generation 24: Best Fitness = 0.284
Generation 25: Best Fitness = 0.285
Generation 26: Best Fitness = 0.291
Generation 27: Best Fitness = 0.303
Generation 28: Best Fitness = 0.300
G